# 1단계: 환경 설정

In [ ]:
# Azure CLI 설치 (이미 설치되어 있다면 생략)
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
# Azure에 로그인
az login
# 리소스 그룹 생성
az group create --name myRAGResourceGroup --location eastus

# 2단계: Azure OpenAI Service 설정

In [ ]:
# Azure OpenAI 리소스 생성
az cognitiveservices account create --name myOpenAI --resource-group myRAGResourceGroup --kind OpenAI --sku S0 --location eastus
# GPT-3.5 모델 배포
az cognitiveservices account deployment create --name myOpenAI --resource-group myRAGResourceGroup --deployment-name gpt35turbo --model-name gpt-35-turbo --model-version "0301" --model-format OpenAI

# 3단계: Azure AI Search 설정

In [ ]:
# Azure AI Search 서비스 생성
az search service create --name mySearchService --resource-group myRAGResourceGroup --sku Basic
# 인덱스 생성 (JSON 파일을 사용하여 인덱스 스키마 정의)
az search index create --name myIndex --service-name mySearchService --resource-group myRAGResourceGroup --definition index-schema.json

# 4단계: 데이터 처리 및 인덱싱
Python 스크립트를 사용하여 데이터를 처리하고 Azure AI Search에 인덱싱합니다.

In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField
from azure.ai.textanalytics import TextAnalyticsClient
from openai import AzureOpenAI
import json
import nest_asyncio
import asyncio
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# 환경 변수에서 필요한 값을 가져옵니다.
TEXT_ANALYTICS_ENDPOINT = os.getenv("TEXT_ANALYTICS_ENDPOINT")
TEXT_ANALYTICS_KEY = os.getenv("TEXT_ANALYTICS_KEY")
OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
OPENAI_KEY = os.getenv("OPENAI_KEY")
SEARCH_ENDPOINT = os.getenv("SEARCH_ENDPOINT")
SEARCH_KEY = os.getenv("SEARCH_KEY")
INDEX_NAME = os.getenv("INDEX_NAME")

# Azure 클라이언트를 초기화합니다.
text_analytics_client = TextAnalyticsClient(endpoint=TEXT_ANALYTICS_ENDPOINT, credential=AzureKeyCredential(TEXT_ANALYTICS_KEY))
openai_client = AzureOpenAI(api_key=OPENAI_KEY, api_version="2023-05-15", azure_endpoint=OPENAI_ENDPOINT)
index_client = SearchIndexClient(endpoint=SEARCH_ENDPOINT, credential=AzureKeyCredential(SEARCH_KEY))
search_client = SearchClient(endpoint=SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=AzureKeyCredential(SEARCH_KEY))

def create_search_index():
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SearchableField(name="keywords", type=SearchFieldDataType.Collection(SearchFieldDataType.String)),
        SearchableField(name="metadata", type=SearchFieldDataType.String)
    ]
    index = SearchIndex(name=INDEX_NAME, fields=fields)
    try:
        index_client.create_index(index)
    except Exception as e:
        print(f"Index creation failed: {e}")

def chunk_document(content, max_chunk_size=500):
    words = content.split()
    chunks = []
    chunk = []
    chunk_size = 0
    for word in words:
        if chunk_size + len(word) + 1 > max_chunk_size:
            chunks.append(" ".join(chunk))
            chunk = []
            chunk_size = 0
        chunk.append(word)
        chunk_size += len(word) + 1
    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

def process_and_index_document(document):
    key_phrase_result = text_analytics_client.extract_key_phrases([document["content"]])
    keywords = key_phrase_result[0].key_phrases if key_phrase_result else []
    keywords_str = json.dumps(keywords)
    chunks = chunk_document(document["content"])
    for i, chunk in enumerate(chunks):
        search_client.upload_documents([{
            "id": f"{document['id']}-{i}",
            "content": chunk,
            "keywords": keywords_str,
            "metadata": json.dumps(document["metadata"])
        }])

# 5단계: Advanced RAG 파이프라인 구현

In [4]:
# 비동기 쿼리 실행 함수 정의
async def run_query(query):
    results = search_client.search(search_text=query, top=2)
    return list(results)

# 여러 쿼리를 병렬로 실행하는 함수 정의
async def run_parallel_queries(queries):
    tasks = [run_query(query) for query in queries]
    return await asyncio.gather(*tasks)

# BM25 점수를 계산하는 함수 정의
def bm25_score(result, query):
    content = result.get("content", "")
    return sum(content.lower().count(term.lower()) for term in query.split())

# 결과를 재정렬하는 함수 정의
def rerank_results(results, query):
    return sorted(results, key=lambda result: bm25_score(result, query), reverse=True)

# 컨텍스트를 생성하는 함수 정의
def generate_context(results):
    return "\n".join([result["content"] for result in results[:3]])

# 응답을 생성하는 함수 정의
def generate_response(query, context):
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    response = openai_client.chat.completions.create(
        model="gpt-35-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# 메인 함수 정의
def main():
    create_search_index()
    
    # 인덱싱할 문서 목록을 정의합니다.
    documents = [
        {"id": "1", "content": "Azure Cognitive Services는 자연어 처리, 컴퓨터 비전, 음성 인식과 같은 강력한 AI 기능을 제공합니다.", "metadata": {"category": "AI", "topic": "Cognitive Services"}},
        {"id": "2", "content": "Azure Search는 애플리케이션에 풍부한 검색 환경을 구축할 수 있도록 지원하는 완전 관리형 검색 서비스입니다.", "metadata": {"category": "Search", "topic": "Azure Search"}},
        {"id": "3", "content": "Azure OpenAI Service는 비즈니스 워크플로우에 GPT-3와 같은 대형 언어 모델을 통합하여 요약 및 감정 분석과 같은 작업을 수행할 수 있게 합니다.", "metadata": {"category": "AI", "topic": "OpenAI Service"}},
        {"id": "4", "content": "Advanced RAG(Retrieval-Augmented Generation)는 문서 검색과 생성형 AI를 결합하여 보다 정확하고 맥락적으로 관련성 있는 응답을 제공합니다.", "metadata": {"category": "AI", "topic": "Advanced RAG"}},
        {"id": "5", "content": "Azure Blob Storage는 이미지, 비디오 및 문서를 포함한 비정형 데이터를 위한 확장 가능하고 비용 효율적인 스토리지 솔루션을 제공합니다.", "metadata": {"category": "Storage", "topic": "Blob Storage"}},
        {"id": "6", "content": "Azure Kubernetes Service(AKS)는 클라우드에서 Kubernetes를 사용하여 컨테이너화된 애플리케이션을 배포 및 관리하는 작업을 간소화합니다.", "metadata": {"category": "DevOps", "topic": "Kubernetes"}},
        {"id": "7", "content": "Azure Functions는 이벤트 중심 애플리케이션을 인프라 관리 없이 구축할 수 있는 서버리스 컴퓨팅 플랫폼을 제공합니다.", "metadata": {"category": "Serverless", "topic": "Azure Functions"}},
        {"id": "8", "content": "Azure Synapse Analytics는 조직이 대규모 데이터 통합 및 빅데이터 분석을 수행할 수 있도록 지원합니다.", "metadata": {"category": "Analytics", "topic": "Synapse"}},
        {"id": "9", "content": "Microsoft Power BI는 데이터를 시각화하고 대화형 대시보드를 생성하여 인사이트를 도출할 수 있도록 합니다.", "metadata": {"category": "Analytics", "topic": "Power BI"}},
        {"id": "10", "content": "Azure DevOps는 개발팀에 버전 관리, CI/CD 및 프로젝트 관리 도구를 제공합니다.", "metadata": {"category": "DevOps", "topic": "DevOps Tools"}},
        {"id": "11", "content": "Azure Security Center는 하이브리드 클라우드 워크로드 전반에 걸쳐 통합된 보안 관리와 고급 위협 방지 기능을 제공합니다.", "metadata": {"category": "Security", "topic": "Security Center"}},
        {"id": "12", "content": "Azure Monitor는 애플리케이션과 인프라에 대한 완전한 관측성을 제공하며 사전적인 문제 해결을 가능하게 합니다.", "metadata": {"category": "Monitoring", "topic": "Azure Monitor"}},
        {"id": "13", "content": "Azure Logic Apps는 워크플로우를 자동화하고 애플리케이션 및 서비스를 원활하게 통합할 수 있도록 합니다.", "metadata": {"category": "Automation", "topic": "Logic Apps"}},
        {"id": "14", "content": "Azure Machine Learning은 데이터 과학자가 기계 학습 모델을 대규모로 구축, 학습, 배포할 수 있도록 합니다.", "metadata": {"category": "AI", "topic": "Machine Learning"}},
        {"id": "15", "content": "Azure Virtual Machines는 Linux와 Windows를 지원하며 주문형 확장 가능한 컴퓨팅 자원을 제공합니다.", "metadata": {"category": "Compute", "topic": "Virtual Machines"}},
        {"id": "16", "content": "Azure Networking 솔루션에는 애플리케이션 연결성과 보안을 향상시키는 Virtual Network, Load Balancer 및 Application Gateway가 포함됩니다.", "metadata": {"category": "Networking", "topic": "Networking Solutions"}},
        {"id": "17", "content": "Microsoft Teams는 Azure Communication Services와 통합되어 원활한 커뮤니케이션 및 협업 기능을 제공합니다.", "metadata": {"category": "Collaboration", "topic": "Teams Integration"}},
        {"id": "18", "content": "Azure Policy는 조직이 Azure 환경 전반에서 규정을 준수하고 리소스를 관리할 수 있도록 합니다.", "metadata": {"category": "Governance", "topic": "Azure Policy"}},
        {"id": "19", "content": "Azure Backup은 데이터를 클라우드에서 간단하고 신뢰할 수 있는 방식으로 백업하고 복구할 수 있도록 지원합니다.", "metadata": {"category": "Backup", "topic": "Backup Solutions"}},
        {"id": "20", "content": "Azure Databricks는 빅데이터 처리 도구와 통합되어 고급 분석 및 기계 학습 기능을 제공합니다.", "metadata": {"category": "Analytics", "topic": "Databricks"}}
    ]


    for doc in documents:
        process_and_index_document(doc)

    user_query = "Advanced RAG의 장점은 무엇인가요?"
    prompt = f"다음의 Input과 관련된 대체 질문 3개를 작성해주세요.:\n\nInput: {user_query}"

    response = openai_client.chat.completions.create(
        model="gpt-35-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    expanded_queries = [q.strip() for q in response.choices[0].message.content.split("\n") if q.strip()]
    queries = [user_query] + expanded_queries[:3]

    # nest_asyncio를 사용하여 이벤트 루프를 중첩할 수 있도록 합니다.
    nest_asyncio.apply()
    parallel_searchresults = asyncio.run(run_parallel_queries(queries))
    reranked_results = [rerank_results(results, user_query) for results in parallel_searchresults]
    context = generate_context(reranked_results[0])  # 첫 번째 쿼리 결과만 사용
    response = generate_response(user_query, context)

    print("User Query:", user_query)
    print("Response:", response)

if __name__ == "__main__":
    main()


Index creation failed: (ResourceNameAlreadyInUse) Cannot create index 'advancedrag' because it already exists.
Code: ResourceNameAlreadyInUse
Message: Cannot create index 'advancedrag' because it already exists.
Exception Details:	(CannotCreateExistingIndex) Cannot create index 'advancedrag' because it already exists.
	Code: CannotCreateExistingIndex
	Message: Cannot create index 'advancedrag' because it already exists.
User Query: Advanced RAG의 장점은 무엇인가요?
Response: Advanced RAG의 가장 큰 장점은 문서 검색과 생성형 AI를 결합하여 보다 정확하고 맥락적으로 관련성 있는 응답을 제공한다는 것입니다. 이를 통해 사용자들은 더욱 빠르게 필요한 정보를 얻을 수 있으며, 더욱 효율적인 결정을 내릴 수 있습니다. 또한, Advanced RAG는 기존의 검색 엔진과 비교하여 보다 정확하고 포괄적인 검색 결과를 제공하여 사용자들의 만족도를 높일 수 있습니다.
